In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import energyfuctions as ef

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [11]:
### create the test set for the submission 

#load the clean data after feature engineering
train = pd.read_pickle('./data/df_all_noremoveC_addStair2_target_addMA.pkl')
train = ef.reduce_mem_usage(train)

# split the test from the train data set
test = train[train['BuildingID'] == 5]
test = test[(test['Time'] >= '2017-01-01') & (test['Time'] <= '2017-12-31')]

#compare with the submission example file to get the difference
subm  = pd.read_csv("data/result.csv").sort_values(by = ["Type","Time"])
subm['Time'] = pd.to_datetime(subm['Time'])
missing = subm.merge(test[["Type","Time"]], how ='left', on=["Type","Time"], indicator = True)
missing_test = missing[missing['_merge'] == 'left_only'].drop('_merge', axis =1)

print(len(missing_test))
missing_test

Mem. usage decreased to 129.65 Mb (18.7% reduction)
46


,Time,Type,Record
8737,2017-12-31 01:00:00,Q,0
8738,2017-12-31 02:00:00,Q,0
8739,2017-12-31 03:00:00,Q,0
8740,2017-12-31 04:00:00,Q,0
8741,2017-12-31 05:00:00,Q,0
8742,2017-12-31 06:00:00,Q,0
8743,2017-12-31 07:00:00,Q,0
8744,2017-12-31 08:00:00,Q,0
8745,2017-12-31 09:00:00,Q,0
8746,2017-12-31 10:00:00,Q,0


In [12]:
# need to backload the weather data for the 2017-12-31 because the data is missing
path = r'./data/shanghai weather_2017.xlsx'
weather_2017 = pd.read_excel(path)
weather_2017['time'] = pd.to_datetime(weather_2017['time'])
weather_2017[weather_2017['time'] >= '2017-12-31']
weather_2017 = weather_2017.rename(columns={'time':'Time', '温度（℃）':'temp', '露点温度（℃）':'dew point', '相对湿度（%）':'relative humidity', '气压（pa）':'air pressure', '风速（m/s）':'wind speed'}, copy=False)
missing_test = missing_test.merge(weather_2017, how = 'left', on = 'Time')


#backload the other created features
missing_time = train[train['Time'] >= '2017-12-31'] 
missing_time = missing_time[missing_time['BuildingID'] == 5]
missing_time.drop(['Time','Record','temp', 'dew point','relative humidity','air pressure','wind speed'], axis =1, inplace= True)
missing_test = missing_test.merge(missing_time, how = 'left', on = 'Type')


test = test.append(missing_test)

print(set(missing_test['Encoded-Type']))

{0, 1}


In [15]:
print(len(subm))
print(len(test))

17520
17520


In [16]:
#modify the test set to fit for the target building profile
train['target'] = np.log1p(train['Record']/train['Area'])
test['BuildingID'] = 20
test['Encoded-Stair1'] = 30
test['Encoded-Stair2'] = 4
test['Area'] = 101806.0
drop_cols = ['HVACType', 'By_day_record', 'continous_value_flag', 'Stair1', 'Stair2']
train.drop(drop_cols, 1, inplace=True)
test.drop(drop_cols, 1, inplace=True)



In [ ]:
# conduct the prediction using the trained model
test_preds = ef.do_RCV_train(train, test, features)

In [ ]:
#generate the submission file
subm["Record"] = test_preds
subm.to_csv(f"./data/ED01E10015_result_1.csv", index=False)